In [1]:
import math

import torch
import torch.nn as nn

from diffusers.optimization import get_scheduler
from tqdm import tqdm

import pandas as pd
import numpy as np

import random

import os
device = "cuda"
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed(0)


/home/bhux/anaconda3/envs/s2pk/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Random seed set as 0


In [2]:
save_location = "/media/bhux/alpha/xsd_mvp/3000e_optimal_cat_normalized/"

# uniform, lg, gaussian

NUMERICAL = 12
CATEGORICAL = 0

NUM_BINS = 15

INFILLING_TYPE = ''
NOISE_TYPE = ''

In [3]:
from utils import remove_outliers, norm, sample_local_gaussian, convert_categorical, unison_shuffled_copies, categorical_norm

PATH = "./data/xtended_data_all.csv"
EMB_PATH = "./data/xtended_emb_all_deberta_pubchem.npy"

f = pd.read_csv(PATH)
drug_embeddings = np.load(EMB_PATH)
smiles = f['Drug'].values
vlists = {
    col: f[col].values for col in f.drop(labels=['Drug'], axis=1).columns[:NUMERICAL] 
}

inmask = remove_outliers([v for _,v in vlists.items()])

smiles = smiles[inmask]
vlists = {
    k: v[inmask] for k,v in vlists.items()
}

vlists = {
    k: norm(v) for k,v in vlists.items()
}

for col in f.drop(labels=['Drug'], axis=1).columns[NUMERICAL:NUMERICAL+CATEGORICAL]:
    vlists[col] = f[col].values[inmask]

nullmask = np.stack([
    np.isnan(v)==False for _,v in vlists.items()
    ], axis=-1)

for col in f.drop(labels=['Drug'], axis=1).columns[:NUMERICAL]:
    vlists[col+"_cat"], vlists[col] = categorical_norm(vlists[col], numbins=NUM_BINS)

dmss = []
for k,v in vlists.items():
    vlists[k], dms = sample_local_gaussian(v, numbins=NUM_BINS)
    dmss.append(dms)
    print(k)


# for col in f.drop(labels=['Drug'], axis=1).columns[NUMERICAL:]:
#     nan = np.isnan(vlists[col])
#     vlists[col] += 1
#     vlists[col][nan] = 0

# dmss = []
# for k,v in vlists.items():
#     dms = get_local_gaussian(v, numbins=50)
#     dmss.append(dms)

dataset = []
for i, gt in enumerate(zip(*[v for _,v in vlists.items()])):
    dataset.append({
        "sm": smiles[i],
        "ft": drug_embeddings[i],
        "ma": nullmask[i],
        "gt": np.array(gt),
        "od": np.array(gt[NUMERICAL:]),
    })
    # print(gt)
    # print(nullmask[i])
    # break

valCount = np.sum(nullmask, axis=0)*0.1
dataset, rcomb = unison_shuffled_copies(dataset, nullmask)
trdataset = []
valdataset = []
for c, d in zip(rcomb, dataset):
    inc = False
    for i, j in enumerate(list(c)):
        if j and valCount[i] > 0:
            valCount[i] -= 1
            inc = True
    if inc:
        valdataset.append(d)
    else:
        trdataset.append(d)

print(len(trdataset))
print(len(valdataset))
print(len(list(vlists.keys())))

Caco2_Wang
Lipophilicity_AstraZeneca
Solubility_AqSolDB
HydrationFreeEnergy_FreeSolv
PPBR_AZ
VDss_Lombardo
Half_Life_Obach
Clearance_Hepatocyte_AZ
Clearance_Microsome_AZ
LD50_Zhu
herg_central_hERG_at_1uM
herg_central_hERG_at_10uM
Caco2_Wang_cat
Lipophilicity_AstraZeneca_cat
Solubility_AqSolDB_cat
HydrationFreeEnergy_FreeSolv_cat
PPBR_AZ_cat
VDss_Lombardo_cat


/home/bhux/research/syndata/utils.py:268: RuntimeWarning: Mean of empty slice.
  m.append(data.mean() if not np.isnan(data.mean()) else low)
/home/bhux/anaconda3/envs/s2pk/lib/python3.12/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/bhux/anaconda3/envs/s2pk/lib/python3.12/site-packages/numpy/_core/_methods.py:227: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/bhux/anaconda3/envs/s2pk/lib/python3.12/site-packages/numpy/_core/_methods.py:184: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/bhux/anaconda3/envs/s2pk/lib/python3.12/site-packages/numpy/_core/_methods.py:219: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Half_Life_Obach_cat
Clearance_Hepatocyte_AZ_cat
Clearance_Microsome_AZ_cat
LD50_Zhu_cat
herg_central_hERG_at_1uM_cat
herg_central_hERG_at_10uM_cat
30264
2864
24


In [4]:
from torch.utils.data import Dataset

class GaucamolDataset(Dataset):
    def __init__(self, dataset) -> None:
        self.dataset = dataset
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]
    
    def update(self, idx, delta):
        item = self.dataset[idx]["gt"]
        self.dataset[idx]["gt"] = item + delta

trainset = GaucamolDataset(trdataset)
valset = GaucamolDataset(valdataset)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=128, shuffle=True)
steps_per_epoch = len(trainset)
DMSS = dmss[:NUMERICAL]

In [5]:
def train(diffusion, ema, gamma, dataloader, optimizer, lr_scheduler, two_noise=False):
    diffusion.train()
    running_loss = 0
    global_step = 0
    for i, batch in enumerate(tqdm(dataloader)):
        ft = batch['ft'].to(device).float()
        gt = batch['gt'].to(device).float()
        od = batch['od'].to(device).long()
        mask = batch['ma'].to(device)
        bs = ft.shape[0]

        optimizer.zero_grad()
        loss_multi, loss_gauss = diffusion.mixed_loss(ft, gt, od, mask, DMSS)

        loss = loss_multi + loss_gauss
        
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        ema.update_params(gamma)
        gamma = ema.update_gamma(global_step)

        running_loss += loss.item()
        global_step += 1
    return running_loss/global_step

In [6]:
from sklearn.metrics import mean_squared_error
import csv
from utils import ohe_to_categories

def evaluate(e, ema, dataloader):
    ema.ema_model.eval()
    before_mse = 0
    running_mse = 0
    global_step = 0
    vals = {}
    device = 'cuda'
    ema.ema_model.to(device)
    with torch.no_grad():
        for i, batch in enumerate(tqdm(dataloader)):
            sm = batch['sm']
            mask = batch['ma'].repeat(1,2)
            ft = batch['ft'].to(device).float()
            gt = batch['gt'].to(device).float()
            od = batch['od'].to(device).long()
            bs = ft.shape[0]

            x_in, generated_ys = ema.ema_model.sample(ft, bs, od, DMSS, clip_sample=True)

            raw_mse = mean_squared_error(gt[mask].flatten().cpu(), x_in[mask].flatten().cpu())
            mse = mean_squared_error(gt[mask].flatten().cpu(), generated_ys[mask].flatten().cpu())

            for s, g in zip(sm, list(generated_ys.cpu().numpy())):
                vals[s] = g
            
            before_mse += raw_mse
            running_mse += mse
            global_step += 1

    with open(save_location+'{}_dict.csv'.format(e), 'w') as csv_file:  
        writer = csv.writer(csv_file)
        for key, value in vals.items():
            writer.writerow([key, value])

    return running_mse / global_step, before_mse / global_step
            

In [7]:
from ema import EMA
lr = 0.00022643741661680812
wd = 0.0004940511856069883
warmup = 50
n_timesteps = 2000
n_inference_timesteps = 150
num_epochs = 30000
update_epochs = 500
update_timesteps = int(num_epochs/update_epochs)
gamma = 0.9739783641481703

In [8]:


from sdt import SDT
from diffusion import GaussianMultinomialDiffusion
torch.set_printoptions(profile="full")


total_num_steps = (steps_per_epoch * num_epochs)

model = SDT(
    time_dim = 64,
    cond_size = 768,
    patch_size = 64,
    y_dim = NUMERICAL+(NUM_BINS+1)*(NUMERICAL+CATEGORICAL),
    dim = 768,
    depth = 12,
    heads = 7,
    mlp_dim = 512,
    dropout =  0.16614739878727047,
    emb_dropout =  0.16614739878727047,
)
model.to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {total_params}")

diffusion = GaussianMultinomialDiffusion(
    num_classes = np.array([(NUM_BINS+1) for _ in range(NUMERICAL+CATEGORICAL)]),
    num_numerical_features = NUMERICAL,
    denoise_fn = model,
    device = device,
)
diffusion.to(device)

ema = EMA(diffusion, gamma, total_num_steps)

optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=wd,
    )

lr_scheduler = get_scheduler(
        "cosine_with_restarts",
        optimizer=optimizer,
        num_warmup_steps=warmup,
        num_training_steps=total_num_steps,
    )

Number of parameters: 40865868
torch.Size([192])


In [ ]:
l = ""
best_mse = 1
loss = 0
for e in range(num_epochs):
    loss = train(diffusion, ema, gamma, trainloader, optimizer, lr_scheduler)
    if (e % 10 == 0) and (e > 0):
        mse, bmse = evaluate(e, ema, valloader)
        print(e, "avgloss {}, avgvalmse {}, beforemse: {}".format(loss, mse, bmse))
        l += "{} avgloss {}, avgvalmse {}, beforemse: {}\n".format(e, loss, mse, bmse)

        if mse < best_mse:
            best_mse = mse
            torch.save({
                'e': e,
                'ema_model': ema.ema_model.state_dict(),
                'model': diffusion.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
            }, save_location+"best_model.pt")
    else:
        print(e, "avgloss {}".format(loss))
        l += "{} avgloss {}\n".format(e, loss)

    # if ((e % update_epochs  == 0) and e > 500):
    #     trainloader = update(int((num_epochs-e) / update_epochs), ema, updateloader, trainset, ns, update_timesteps)

    with open(save_location+'output.txt', 'w') as file:
        file.write(l)

100%|██████████| 237/237 [00:11<00:00, 20.01it/s]


0 avgloss 0.6395241586719385


100%|██████████| 237/237 [00:10<00:00, 23.49it/s]


1 avgloss 0.6381343012117635


100%|██████████| 237/237 [00:10<00:00, 23.70it/s]


...

100%|██████████| 237/237 [00:10<00:00, 21.73it/s]


99 avgloss 0.6155470764335198


100%|██████████| 23/23 [00:39<00:00,  1.70s/it]


100 avgloss 0.6202309620782795, avgvalmse 11.37491571187731, beforemse: 14.607922554016113


 47%|████▋     | 111/237 [00:05<00:06, 19.98it/s]